In [1]:
#导入一系列包
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
file_path2='D:/files/data/stopwordlisteng.TXT'
stopworddic1=stop_words
data=[]
with open(file_path2, "r") as f:
    for line in f.readlines():
        line = line.strip('\n')  #去掉列表中每一个元素的换行符
        data.append(line)
        #print(line)
for i in range(len(data)):
    stopworddic1.append(data[i])

fh=[',','(',')','e','.','g','-']

for i in range(1000):
    fh.append(str(i))

for i in range(len(fh)):
    stopworddic1.append(fh[i])

In [4]:
# Import Dataset
file_path='D:/files/data/finaldata/DII.xls'
df=pd.read_excel(file_path)
data = df.keycontent.values.tolist()

In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [7]:
stop_words = stopworddic1
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
data_words
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [9]:
#data_words_nostops

In [10]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [11]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=12, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.5,
                                           per_word_topics=True)

In [12]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.116022  0.057057       1        1  15.468048
8      0.128816 -0.032275       2        1  13.736818
11     0.134215  0.085418       3        1  12.305344
3     -0.241529 -0.001401       4        1   8.366460
0      0.204171 -0.079092       5        1   7.636258
1      0.191046 -0.029956       6        1   7.410116
4     -0.040824  0.252398       7        1   7.290143
10    -0.118648 -0.218026       8        1   6.280434
6     -0.215641  0.069513       9        1   5.972764
7     -0.084287  0.130445      10        1   5.543003
9     -0.147378 -0.159687      11        1   5.355125
5      0.074037 -0.074392      12        1   4.635488, topic_info=               Term          Freq         Total Category  logprob  loglift
term                                                                      
39           system  16704.000000  16704.000000  Default  30.0000  30.0000
373          device   6702.000000   6702.000000  Default  29.0000  29.0000
81            drive   6079.000000   6079.000000  Default  28.0000  28.0000
9           control  10407.000000  10407.000000  Default  27.0000  27.0000
80            datum   7047.000000   7047.000000  Default  26.0000  26.0000
238          object   4744.000000   4744.000000  Default  25.0000  25.0000
41          vehicle  45951.000000  45951.000000  Default  24.0000  24.0000
2        autonomous  23200.000000  23200.000000  Default  23.0000  23.0000
74             unit   4112.000000   4112.000000  Default  22.0000  22.0000
109           image   3622.000000   3622.000000  Default  21.0000  21.0000
116          method  10804.000000  10804.000000  Default  20.0000  20.0000
82          driving   2921.000000   2921.000000  Default  19.0000  19.0000
93           sensor   6485.000000   6485.000000  Default  18.0000  18.0000
37           signal   2954.000000   2954.000000  Default  17.0000  17.0000
151       determine   6346.000000   6346.000000  Default  16.0000  16.0000
402            step   3096.000000   3096.000000  Default  15.0000  15.0000
560           route   2709.000000   2709.000000  Default  14.0000  14.0000
293          module   2537.000000   2537.000000  Default  13.0000  13.0000
1158          light   2480.000000   2480.000000  Default  12.0000  12.0000
1866      computing   2790.000000   2790.000000  Default  11.0000  11.0000
113        location   3321.000000   3321.000000  Default  10.0000  10.0000
140            mode   1952.000000   1952.000000  Default   9.0000   9.0000
727          driver   1920.000000   1920.000000  Default   8.0000   8.0000
998            user   2775.000000   2775.000000  Default   7.0000   7.0000
341            road   2214.000000   2214.000000  Default   6.0000   6.0000
878       apparatus   2000.000000   2000.000000  Default   5.0000   5.0000
336            lane   1936.000000   1936.000000  Default   4.0000   4.0000
195        automate   1856.000000   1856.000000  Default   3.0000   3.0000
71           target   1499.000000   1499.000000  Default   2.0000   2.0000
322         network   2067.000000   2067.000000  Default   1.0000   1.0000
...             ...           ...           ...      ...      ...      ...
2392        history     56.967193     57.910656  Topic12  -6.2465   3.0550
2786       stopping     56.204837     57.148335  Topic12  -6.2600   3.0548
2273         encode     51.144733     52.088238  Topic12  -6.3543   3.0531
81            drive   5718.252730   6079.812704  Topic12  -1.6375   3.0101
147          switch    518.884447    573.592849  Topic12  -4.0373   2.9712
455           human    440.178977    691.543884  Topic12  -4.2018   2.6197
33           period    234.815894    353.191051  Topic12  -4.8302   2.6632
259      capability    208.596791    319.993802  Topic12  -4.9486   2.6435
913       condition    470.638581   1025.739069  Topic12  -4.1349   2.2924
352   automatically    240.736094    497.399047  To

In [13]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.080*"method" + 0.063*"determine" + 0.061*"step" + 0.053*"vehicle" + '
  '0.038*"base" + 0.035*"involve" + 0.024*"action" + 0.019*"motion" + '
  '0.018*"autonomous" + 0.015*"risk"'),
 (1,
  '0.094*"vehicle" + 0.058*"route" + 0.050*"location" + 0.047*"road" + '
  '0.041*"lane" + 0.035*"traffic" + 0.031*"method" + 0.031*"determine" + '
  '0.027*"travel" + 0.019*"autonomous"'),
 (2,
  '0.208*"vehicle" + 0.133*"autonomous" + 0.077*"control" + 0.020*"operation" '
  '+ 0.019*"system" + 0.016*"speed" + 0.014*"path" + 0.014*"description" + '
  '0.013*"base" + 0.012*"include"'),
 (3,
  '0.024*"wheel" + 0.023*"brake" + 0.022*"seat" + 0.021*"sensor" + '
  '0.020*"assembly" + 0.018*"motor" + 0.015*"frame" + 0.015*"vehicle" + '
  '0.015*"provide" + 0.015*"surface"'),
 (4,
  '0.288*"system" + 0.057*"computing" + 0.052*"vehicle" + 0.040*"automate" + '
  '0.015*"comprise" + 0.013*"operate" + 0.012*"description" + 0.012*"block" + '
  '0.012*"multiple" + 0.011*"provide"'),
 (5,
  '0.194*"drive"

In [12]:
from pandas.core.frame import DataFrame
texts1=DataFrame(texts)
topics = [lda_model[corpus[i]] for i in range(len(texts1))]

In [13]:
j=0
def topics_document_to_dataframe(topics_document, num_topics):
    global j
    j=j+1
    res = pd.DataFrame(columns=range(num_topics))
    test_2 = pd.DataFrame()  
    for i in range(12):
        test_2.loc[str(j),i]=0
    res1=test_2
    #res.index=0
    #res.ix[]
    for topic_weight in topics_document:
        #print(topic_weight)
        #print(topic_weight[1])
       # print(res)
        #print(res.ix[0, topic_weight[0]])
        res1.loc[str(j),topic_weight[0]] = topic_weight[1]
    return res1

In [14]:
document_topic=[]
document_topic = pd.concat([topics_document_to_dataframe(topics_document[0], 12) for topics_document in topics])

In [15]:
document_topic.to_excel('D:/files/data/finaldata/diidataframe.xls', index=False)